In [1]:
import numpy as np
import pandas as pd

## Preprocessing

In [2]:
import json
import re

In [3]:
def parse(path):
    with open(path, 'r') as f:
        text_data = f.read()
    for l in text_data.split('\n'):
        yield l

In [4]:
def getDF(path):
    dicts = []
    for d in parse(path):
        #print(d)
        try:
            dicts.append(json.loads(d))
        except:
            print(d)
            print(type(d))
    return pd.DataFrame(dicts)

In [16]:
import nltk
from nltk.tokenize import word_tokenize

In [100]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [98]:
def my_normalizer(string):
    strr = string.lower() #нижний регистр
    strr = strr.strip() #выкидываем пробельные символы в начале и конце текста
    strr = re.sub('\s+', ' ', strr) #заменяем последовательности из пробельных символов на обычные пробелы
    strr = re.sub("[^A-Za-z]+", ' ', strr) #выкидываем не буквы 
    tok_str = word_tokenize(strr)
    stopWords = set(stopwords.words('english')) #список стоп-слов
    Lemmatizer = WordNetLemmatizer()
    output_str = []
    for word in tok_str:
        if word not in stopWords: #выкидывае стоп-слова
            output_str.append(Lemmatizer.lemmatize(word)) #добавляем лемматизированое слово
    return output_str

In [5]:
data = getDF('Data/Movies_and_TV_5.json')


<class 'str'>


In [9]:
data.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,ADZPIG9QOCDG5,0005019281,"Alice L. Larson ""alice-loves-books""","[0, 0]",This is a charming version of the classic Dick...,4.0,good version of a classic,1203984000,"02 26, 2008"
1,A35947ZP82G7JH,0005019281,Amarah Strack,"[0, 0]",It was good but not as emotionally moving as t...,3.0,Good but not as moving,1388361600,"12 30, 2013"
2,A3UORV8A9D5L2E,0005019281,Amazon Customer,"[0, 0]","Don't get me wrong, Winkler is a wonderful cha...",3.0,Winkler's Performance was ok at best!,1388361600,"12 30, 2013"
3,A1VKW06X1O2X7V,0005019281,"Amazon Customer ""Softmill""","[0, 0]",Henry Winkler is very good in this twist on th...,5.0,It's an enjoyable twist on the classic story,1202860800,"02 13, 2008"
4,A3R27T4HADWFFJ,0005019281,BABE,"[0, 0]",This is one of the best Scrooge movies out. H...,4.0,Best Scrooge yet,1387670400,"12 22, 2013"


In [10]:
data.iloc[0]['reviewText']

'This is a charming version of the classic Dicken\'s tale.  Henry Winkler makes a good showing as the "Scrooge" character.  Even though you know what will happen this version has enough of a change to make it better that average.  If you love A Christmas Carol in any version, then you will love this.'

In [ ]:
%%time
norm_rev = pd.Series(data['reviewText'].apply(my_normalizer))

Сохраним

In [15]:
#with open('norm_texts.txt', 'w') as f:
#    for x in norm_rev:
#        f.write(' '.join(x) + '\n')

Загрузим

In [9]:
#norm_data = []
#with open('norm_texts.txt', 'r') as f:
#    for line in f:
#        norm_data += [line.strip()]

In [12]:
#norm_df = pd.DataFrame(data={'review': [x.split() for x in norm_data]})

In [ ]:
#norm_df = pd.DataFrame([], columns=['data'])
#k = 0
#with open('norm_texts.txt', 'r') as f:
#    for line in f:
#        norm_df.loc[k, 'data'] = [line.strip()]
#        k += 1

## Separation on train valid test

Возьмем 70% данных для тренировки и по 15% оставим на валидацию и тест

In [20]:
train_ind, valid_ind, test_ind = np.split(np.random.permutation(norm_df.index), [int(len(norm_df)*0.7), int(len(norm_df)*0.85)])

Сохраним индексы

In [25]:
#np.save('train_ind', train_ind)
#np.save('valid_ind', valid_ind)
#np.save('test_ind', test_ind)

In [15]:
len(test_ind)/len(norm_df)

0.15000002945450838

In [15]:
#train_df = norm_df.iloc[train_ind]
#valid_df = norm_df.iloc[valid_ind]
#test_df = norm_df.iloc[test_ind]

In [46]:
#train_df.to_csv('train_df.csv')
#valid_df.to_csv('valid_df.csv')
#test_df.to_csv('test_df.csv')

In [2]:
#train_df = pd.read_csv('train_df.csv', index_col=0)
#valid_df = pd.read_csv('valid_df.csv', index_col=0)
#test_df = pd.read_csv('test_df.csv', index_col=0)

/Users/danilculkov/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [16]:
#train_df.head()

,review,overall
855191,"[movie, poor, quality, actor, never, heard, li...",2.0
473387,"[great, group, costars, violent, child, great,...",4.0
682386,"[james, franco, may, resemble, james, dean, ho...",2.0
710604,"[well, like, long, ago, old, chump, mine, deci...",5.0
715706,"[review, series, apperently, made, rigid, fan,...",5.0


### GENSIM

In [17]:
#import gensim as gs

Попробуем выделить некоторые очевидные биграмы в нашем тесте (аля Имя + Фамилия), для того чтобы сразу воспринимать их как некоторые единицы.

In [35]:
#from gensim.test.utils import common_texts

In [36]:
#from gensim.models.phrases import Phrases, Phraser

In [74]:
#%%time
#phrases = Phrases(train_df['review'], min_count=int(len(train_df)*0.001))
#bigram = Phraser(phrases)

CPU times: user 6min 33s, sys: 45.8 s, total: 7min 19s
Wall time: 8min 34s


Эмпиризм значения параметра min_count следующий: если биграм встречается (очень грубо говоря) хотя бы в 0.1% отзывов, то мы рассматриваем его всерьез. При выборе небольших значений этого параметра (+- 100) в него попадают словосочетания, которые на самом деле хотелось бы выявлять моделью.

In [75]:
#%%time
#train_df_with_bigram = train_df['review'].apply(lambda x: bigram[x])

CPU times: user 4min 13s, sys: 2min 17s, total: 6min 31s
Wall time: 9min 26s


In [76]:
#train_df['with_bigram'] = train_df_with_bigram

/Users/danilculkov/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
#%%time
#valid_df['with_bigram']  = valid_df['review'].apply(lambda x: bigram[x])

CPU times: user 47.6 s, sys: 7.33 s, total: 55 s
Wall time: 1min 3s


/Users/danilculkov/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [84]:
#train_df.drop(columns=['with_bigram'], inplace=True)
#valid_df.drop(columns=['with_bigram'], inplace=True)

/Users/danilculkov/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


__В последствии, я отказался от этой идеи, но все же стирать ничего не стал__

## Own Word2Vec model

Попробуем сначала натренить свою собственную модель word2vec.

In [2]:
from gensim.models import Word2Vec

In [5]:
import multiprocessing

In [24]:
model = Word2Vec(size=250, window=5, min_count=1, workers=10)

In [32]:
%%time
model.build_vocab(train_df['review'])

CPU times: user 2min 12s, sys: 48.5 s, total: 3min
Wall time: 3min 27s


In [33]:
%%time
model.train(train_df['review'], total_examples=len(train_df), epochs=20)

CPU times: user 2h 1min 19s, sys: 1min 32s, total: 2h 2min 51s
Wall time: 24min 14s


(1932365495, 2023813520)

Мдааааа...

In [34]:
model.init_sims(replace=True)

In [35]:
model.wv.most_similar(positive='depression')

[('alcoholism', 0.5647395253181458),
 ('despair', 0.5455694794654846),
 ('poverty', 0.5399671792984009),
 ('loneliness', 0.5275277495384216),
 ('disillusionment', 0.5142602324485779),
 ('hopelessness', 0.5126552581787109),
 ('joblessness', 0.5081775784492493),
 ('prohibition', 0.48855435848236084),
 ('recession', 0.48155638575553894),
 ('malaise', 0.47719451785087585)]

И правда ведь!

In [93]:
model.wv.most_similar(positive='traditional')

[('conventional', 0.677680253982544),
 ('traditionally', 0.5697474479675293),
 ('westernized', 0.5693082213401794),
 ('typical', 0.5363076329231262),
 ('classical', 0.5356478691101074),
 ('contemporary', 0.5137262344360352),
 ('modern', 0.5043244361877441),
 ('tradition', 0.5028184056282043),
 ('traditonal', 0.49846452474594116),
 ('style', 0.4758468568325043)]

In [94]:
model.wv.most_similar(positive='america')

[('country', 0.6292040944099426),
 ('nation', 0.6078619956970215),
 ('american', 0.6031495332717896),
 ('usa', 0.599901556968689),
 ('korea', 0.5874038934707642),
 ('africa', 0.5859670042991638),
 ('europe', 0.5397226810455322),
 ('nanook', 0.519948422908783),
 ('japan', 0.5129011869430542),
 ('britain', 0.5075209736824036)]

In [95]:
model.wv.most_similar(positive='time')

[('hour', 0.5217479467391968),
 ('minute', 0.5182461142539978),
 ('awhile', 0.5045113563537598),
 ('year', 0.49275457859039307),
 ('day', 0.49242666363716125),
 ('twice', 0.49231043457984924),
 ('since', 0.4696785509586334),
 ('week', 0.46965134143829346),
 ('movie', 0.460769921541214),
 ('month', 0.4499945640563965)]

Блин, так круто, аж наиграться не могу.... 

In [96]:
model.wv.similarity('god', 'devil')

0.35250312

И правда не очень похожи)

In [97]:
model.wv.similarity('mother', 'father')

0.8487515

In [99]:
model.wv.most_similar(positive=['mother', 'man'], negative=['woman'])

[('father', 0.7148435115814209),
 ('dad', 0.6104788780212402),
 ('grandfather', 0.5882338285446167),
 ('son', 0.5732370615005493),
 ('uncle', 0.5580236911773682),
 ('brother', 0.4667826294898987),
 ('grandmother', 0.4646051526069641),
 ('grandpa', 0.46417585015296936),
 ('stepdad', 0.4605330228805542),
 ('mom', 0.45149534940719604)]

Ура! Вроде пока не очень-то все и плохо.

Сохраню ка я свою модельку, не зря же столько старалась!

In [36]:
#model.wv.save_word2vec_format('model_1.bin', binary=True)

In [37]:
model.save('model_1_v2')

## Bidirectional Simple RNN + our Word2Vec

Теперь мы попробуем построить простую RNN, используя уже полученные нашим ворд2веком (натренированным на train) эмбеддинги, для предсказания оценки.

Загрузка модели

In [3]:
W2V_model = Word2Vec.load("model_1_v2")

In [4]:
vocab_size = len(W2V_model.wv.vocab)
embedding_size = 250

In [5]:
embedding_matrix = np.zeros((vocab_size, embedding_size))
for i in range(vocab_size):
    embedding_vector = W2V_model.wv[W2V_model.wv.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [3]:
from tensorflow import keras

In [4]:
from tensorflow.keras import layers

In [5]:
from tensorflow.keras import preprocessing

Загрузим индексы

In [6]:
train_ind = np.load('train_ind.npy')
valid_ind = np.load('valid_ind.npy')
test_ind = np.load('test_ind.npy')

In [10]:
# всего примеров
print(len(train_ind) + len(valid_ind) + len(test_ind))

1697533


Ответы

In [7]:
overall = pd.read_csv('overall.csv', index_col=0)
overall['overall'] = overall['overall'].apply(int)

/Users/danilculkov/opt/anaconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


### Немного отвлечемся...

Немного отвлечемся и посмотрим на наши данные, а именно на то, как они распределены относительно целевых классов.

In [12]:
for i in range(1,6):
    print('Overall {}: {} samples = {}%'.format(i, np.sum(overall['overall']==i), round(np.sum(overall['overall']==i)*100/len(overall), 3)))

Overall 1: 104219 samples = 6.139%
Overall 2: 102410 samples = 6.033%
Overall 3: 201302 samples = 11.859%
Overall 4: 382994 samples = 22.562%
Overall 5: 906608 samples = 53.407%


Выходит, что в целом положительных отзывов в исходных данных значительно больше. Получается, что если бы мы всем предсказывали 5*, то получилась бы уже неплохая модель (шутка). Это не очень хорошо, ведь модель и правда может подогнаться под такой дисбалланс, что не даст нам требуемого результата, а будет лишь "позитивно" относиться ко всем отзывам (именно это у меня и получилось при первых попытках).

Согласно гуглу, существует несколько основных подходов к работе с дисбаллансированными классами:

1) Андерсэмплинг (придется что-то выкинуть)

2) Оверсэмплинг (у нас и так больше миллиона отзывов в треине, а тут придется еще добивть)

3) Навешивание весов классам в керасе (тут требуются определенные эвристики по выбору этих весов)

Я решил остановиться на последнем.

In [13]:
#размер треина
len(train_ind)

1188273

Для экспериментов по выбору весов классов попробуем оставить первые 50к отзывов, чтобы уменьшить время тренировки.

In [11]:
mini_train_ind = train_ind[:100000]

In [86]:
tiny_train_ind = train_ind[:10000]

In [121]:
for i in range(1,6):
    print(i, np.sum(overall.loc[tiny_train_ind, 'overall']==i)/len(tiny_train_ind))

1 0.0607
2 0.0608
3 0.1164
4 0.2223
5 0.5398


Получилось вполне себе репрезентативненько.

Далее (немного заглядывая вперед, чтобы не загромождать notebook), для небольшого набора весов я немного потренил (ненастроенные) сети на мини треине и вот что получилось на валидации за 5 эпох:

In [83]:
result_weights

,class_weights,loss,accuracy
0,"{0: 1, 1: 1, 2: 0.9, 3: 0.75, 4: 0.5}",1.129692,0.54498
1,"{0: 1, 1: 1, 2: 0.9, 3: 0.8, 4: 0.6}",1.056613,0.56480
2,"{0: 1, 1: 1, 2: 0.9, 3: 0.8, 4: 0.7}",1.013923,0.59174
3,"{0: 1, 1: 1, 2: 0.9, 3: 0.85, 4: 0.75}",1.033361,0.57784
4,"{0: 1, 1: 1, 2: 0.9, 3: 0.9, 4: 0.8}",1.046519,0.56732
5,"{0: 1, 1: 1, 2: 0.95, 3: 0.95, 4: 0.9}",1.057140,0.56818
6,"{0: 1, 1: 1, 2: 1, 3: 1, 4: 1}",1.099053,0.56232


Соответственно, оставляем вариант с наилучшим результатом хотя бы в таком приближении.

In [19]:
class_weights = {0: 1, 1: 1, 2: 0.9, 3: 0.8, 4: 0.7}

### Вернемся обратно

Подготовим данные. Функция чтения и кодирования нормализованного набора ревьюшек

In [8]:
def my_encoding_reviews(vocabulary, indexes_of_reviews):
    encoded_reviews = []
    target = []
    with open('norm_texts.txt', 'r') as f:
        k = 0
        for line in f:
            if k in indexes_of_reviews:
                sen = line.split()
                encoded_review = []
                for x in sen:
                    if x in vocabulary.keys():
                        encoded_review += [vocabulary[x].index]
                encoded_reviews.append(encoded_review)
                target += [int(overall.loc[k, 'overall'])]
            k+=1
            if k % 250000 == 0:
                print("Осталось прочитать:", len(overall) - k)
    return [encoded_reviews, target]

Загружаем и кодируем треин

In [23]:
%%time
[encoded_train, train_y] = my_encoding_reviews(W2V_model.wv.vocab
                                                    , train_ind)

Осталось прочитать: 1447533
Осталось прочитать: 1197533
Осталось прочитать: 947533
Осталось прочитать: 697533
Осталось прочитать: 447533
Осталось прочитать: 197533
CPU times: user 13min 13s, sys: 4.47 s, total: 13min 18s
Wall time: 13min 18s


Обрезаем ревьюшки по среднему размеру, так обучение проходит в десятки раз быстрее.

In [24]:
max_review_len = 1
sum_review_len = 0
for x in encoded_train:
    sum_review_len += len(x)
    if len(x) > max_review_len:
        max_review_len = len(x)
avg_review_len = sum_review_len/len(encoded_train)
padded_train = preprocessing.sequence.pad_sequences(encoded_train
                                                    , maxlen = round(avg_review_len)
                                                    , padding='post')


Теперь посмотрим непосредственно на саму нейронку. Я решил (следуя заданию) пока остановиться на простой двувунаправленной рекуррентной сети. Касательно числа нейронов в скрытых слоях: в случаях когда число нейронов скрытых слоев меньше - сети требуется искать меньше параметров, поэтому учится она по-быстрее, однако, мне кажется, немного опромедчиво делать небольшое число нейронов, ведь это наверняка лишь уменьшит гибкость сети. Но и очень много, наверное, тоже не нужно. Короче говоря, тут мне не совсем пока понятно, как нужно действовать. Оставлю пока 250 штук.В качестве оптимизирующего алгоритма берем 'adam' (в интернетах он пропогандируется как один из лучших). 

In [127]:
#Linear stack of layers.
RNN_model = keras.Sequential()
RNN_model.add(layers.Embedding(input_dim=vocab_size
                               , output_dim=embedding_size
                               , input_length=round(avg_review_len)
                               , weights=[embedding_matrix]
                               , trainable=False
                               , mask_zero=True))
RNN_model.add(layers.Bidirectional(layers.SimpleRNN(units=250, activation='relu')))
RNN_model.add(layers.Dense(units=5, activation='softmax'))
RNN_model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, clipnorm=0.5)
                  , loss='sparse_categorical_crossentropy'
                  , metrics=['accuracy'])

RNN_model.summary()

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 85, 250)           146464000 
_________________________________________________________________
bidirectional_11 (Bidirectio (None, 500)               250500    
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 2505      
Total params: 146,717,005
Trainable params: 253,005
Non-trainable params: 146,464,000
_________________________________________________________________


При построении модели я столкнулся с казалось бы маленькой, но принеприятной для меня проблемой - аргументом mask_zero в слое Emdedding. Суть в том, что мой ворд2век присвоил этот индекс слову 'movie' и он как бы уже занят, однако заполнять здесь паддинги чем-то другим не выйдет, ибо аргумент mask_zero позволяет рассматривать лишь нули как пропуски. Чтобы справиться с этим, я пытался найти как можно избежать присвоения этого индекса ворд2веком, пытался добавить слой маскинг (который, как я понял, так сказать, не совместим со слоем Embedding), что в итоге ни к чему не привело. В конечно счете, я решил забить, ограничившись mask_zero=True (слово movie как бы все равно не имеет особого эмоционального окраса, так что пофиг).

Бэтч-сайз я решил взять побольше, модель обучается так по-быстрее, да и поиск оптимума в целом более стабильный.

In [133]:
%%time
RNN_model.fit(padded_train, np.asarray(train_y)-1
              , epochs=5
              , batch_size=1024
              , verbose=1
              , validation_data = (padded_valid[:50000], np.asarray(valid_y)[:50000]-1)
              , class_weight=class_weights
              , workers=10
              , use_multiprocessing=True)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 1188273 samples, validate on 50000 samples
Epoch 1/5
1188273/1188273 [==============================] - 1188s 999us/sample - loss: 0.6926 - accuracy: 0.6544 - val_loss: 0.6658 - val_accuracy: 0.6682
Epoch 2/5
1188273/1188273 [==============================] - 1081s 910us/sample - loss: 0.6863 - accuracy: 0.6574 - val_loss: 0.6661 - val_accuracy: 0.6667
Epoch 3/5
1188273/1188273 [==============================] - 1198s 1ms/sample - loss: 0.6806 - accuracy: 0.6603 - val_loss: 0.6644 - val_accuracy: 0.6701
Epoch 4/5
1188273/1188273 [==============================] - 1132s 952us/sample - loss: 0.6761 - accuracy: 0.6620 - val_loss: 0.6917 - val_accuracy: 0.6465
Epoch 5/5
1188273/1188273 [==============================] - 1169s 984us/sample - loss: 0.6718 - accuracy: 0.6639 - val_loss: 0.6737 - val_accuracy: 0.6691
CPU times: user 9h 28min 16s, sys: 4h 25min 5s, total: 13h 53min 21s
Wall time: 1h 36min 8s


In [138]:
RNN_model.save_weights('Models/RNN_model_v1_relu_adam_0.001_10epochs', save_format='tf')

In [111]:
#RNN_model.load_weights('Models/RNN_model_v1_tanh_adam_0.001_20epochs')

Посмотрим, что у нас будет на валидации. Оттуда выкинем незнакомые слова.

In [38]:
%%time
[encoded_valid, valid_y] = my_encoding_reviews(W2V_model.wv.vocab, valid_ind)

Осталось прочитать: 1447533
Осталось прочитать: 1197533
Осталось прочитать: 947533
Осталось прочитать: 697533
Осталось прочитать: 447533
Осталось прочитать: 197533
CPU times: user 2min 20s, sys: 2.37 s, total: 2min 23s
Wall time: 2min 23s


In [39]:
padded_valid = preprocessing.sequence.pad_sequences(encoded_valid
                                                        , maxlen = round(avg_review_len)
                                                        , padding='post')

In [24]:
# здесь я фиксировал результаты обучения
results = pd.DataFrame([], columns=['description','epochs', 'train_loss', 'train_accuracy', 'valid_loss', 'valid_accuracy'])
results.loc[len(results)] = ['tanh + sgd (lr=0.001)', 10, 0.8230, 0.5933, 1.0011,0.5854]
results.loc[len(results)] = ['tanh + adam (lr=0.0005)', 10, 0.7345, 0.6344, 0.9041,0.6268]
results.loc[len(results)] = ['tanh + adam (lr=0.0005)', 20, 0.7090, 0.6466, 0.7349,0.6309]
results.loc[len(results)] = ['tanh + adam (lr=0.001)', 10, 0.7342, 0.6341, 0.8883,0.6306]
results.loc[len(results)] = ['tanh + adam (lr=0.001)', 20, 0.7128, 0.6438, 0.8755,0.6373]
results.loc[len(results)] = ['tanh + adam (lr=0.001)', 25, 0.7070, 0.6468, 0.8788,0.6356]
results.loc[len(results)] = ['relu + adam (lr=0.0005)' ,10, 0.6828, 0.6603, 0.8461,0.6513]
results.loc[len(results)] = ['relu + adam (lr=0.0005)', 20, 0.6319, 0.6810, 0.8652,0.6485]
results.loc[len(results)] = ['relu + adam (lr=0.001)', 5, 0.6998, 0.6515, 0.8535,0.6469]
results.loc[len(results)] = ['relu + adam (lr=0.001)', 10, 0.6718, 0.6639, 0.8590, 0.6471]

В таблице ниже приводятся сравнительные результы моделей с разными функциями активации и лернинг рейтом.

In [136]:
results

,description,epochs,train_loss,train_accuracy,valid_loss,valid_accuracy
0,tanh + sgd (lr=0.001),10,0.8230,0.5933,1.0011,0.5854
1,tanh + adam (lr=0.0005),10,0.7345,0.6344,0.9041,0.6268
2,tanh + adam (lr=0.0005),20,0.7090,0.6466,0.7349,0.6309
3,tanh + adam (lr=0.001),10,0.7342,0.6341,0.8883,0.6306
4,tanh + adam (lr=0.001),20,0.7128,0.6438,0.8755,0.6373
5,tanh + adam (lr=0.001),25,0.7070,0.6468,0.8788,0.6356
6,relu + adam (lr=0.0005),10,0.6828,0.6603,0.8461,0.6513
7,relu + adam (lr=0.0005),20,0.6319,0.6810,0.8652,0.6485
8,relu + adam (lr=0.001),5,0.6998,0.6515,0.8535,0.6469
9,relu + adam (lr=0.001),10,0.6718,0.6639,0.8590,0.6471


Таким образом, в случае relu-активации уже после 10й эпохи качество модели на валидации перестает расти, соотствественно, пошло переобучение, причем сама модель "выучилась" быстрее всех. В случае с tanh-активацией аналогичное происходит после 20й эпохи. Примечательно, что выставление больших значений learning rate (0.002+) "убивает" модель во время обучения (проверено). Ограничение на норму градиента (клипнорм) также сильно помогло, без него модель ломается даже с lr=0.001 (и, возможно, меньшими рейтами). Последнее возможно обойти и без ограничения на норму - заменой оптимизатора на SGD, однако скорость обучения в таком случае сильно снижается (что также видно из таблицы), ну или нужно подобрать всякие параметры типа момента или подрубить Нестерова.

__Итого__: лучшую оценку на валидаци получается модель с relu-активацией (хоть я ее и не хотел брать), оптимизированная при помощи adam с lr=0.001 на 10 эпохах.

In [134]:
loss, accuracy = RNN_model.evaluate(padded_valid, np.asarray(valid_y)-1, verbose=1)

254630/254630 [==============================] - 130s 509us/sample - loss: 0.8590 - accuracy: 0.6471


Посмотрим на матрицу ошибок лучшей модели.

In [9]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [137]:
%%time
valid_pred_ = RNN_model.predict(padded_valid)

CPU times: user 12min 41s, sys: 3min 42s, total: 16min 24s
Wall time: 1min 54s


In [139]:
valid_pred = np.argmax(valid_pred_, axis=1)

In [141]:
print(confusion_matrix(valid_y, valid_pred+1, labels=np.arange(1,6)))

[[ 11145   1665    833    160   1803]
 [  4956   4231   3290    688   2077]
 [  2576   3377  11048   5506   7807]
 [  1453    893   6433  15118  33506]
 [  2077    438   2659   7661 123230]]


Нууу, почти диагонально.

In [142]:
foo = 0
for i in range(5):
    for j in range(5):
        if np.abs(i-j)>1:
            foo += confusion_matrix(valid_y, valid_pred+1, labels=np.arange(1,6))[i][j]
print("Доля классификацей с ошибкой оценки не более чем на 1: {} %".format(round(100*(1-foo/len(valid_ind)), 2)))

Доля классификацей с ошибкой оценки не более чем на 1: 90.79 %


In [143]:
print(classification_report(valid_y, valid_pred+1, labels=np.arange(1,6)))

              precision    recall  f1-score   support

           1       0.50      0.71      0.59     15606
           2       0.40      0.28      0.33     15242
           3       0.46      0.36      0.40     30314
           4       0.52      0.26      0.35     57403
           5       0.73      0.91      0.81    136065

    accuracy                           0.65    254630
   macro avg       0.52      0.51      0.50    254630
weighted avg       0.62      0.65      0.62    254630



## Bidirectional Simple RNN + Word2Vec from the Internet

Попробуем теперь использовать взятый из Интернета натреннированный Word2Vec.

Источник: [ссылка](https://code.google.com/archive/p/word2vec/) . Модель - __word2vec-google-news-300__.

Также я хотел использовать модели с выглядящего надежнее [vectors.nlpl.eu](http://vectors.nlpl.eu/explore/embeddings/en/models/), но они там PoS-тэггированы, соотвественно, каждое слово имеет "добавку" вида 'table_NOUN'. Получается, нужна некоторая адаптация под наш случай, ибо одному слову (например, dance) могут соотвествовать несколько векторов и таким образом, надо их как-то усреднять чтоли или что-то такое.

In [10]:
from gensim import models

In [11]:
Google_W2V = models.KeyedVectors.load_word2vec_format('GoogleNews.bin', binary=True)

In [12]:
vocab_size_google, embedding_size_google = Google_W2V.vectors.shape

In [13]:
embedding_matrix_google = np.zeros((vocab_size_google, embedding_size_google))
for i in range(vocab_size_google):
    embedding_vector = Google_W2V[Google_W2V.index2word[i]]
    if embedding_vector is not None:
        embedding_matrix_google[i] = embedding_vector

Теперь нам нужно по-новому закодировать треин:

In [14]:
%%time
[encoded_train_google, train_y] = my_encoding_reviews(Google_W2V.vocab
                                                    , train_ind)

Осталось прочитать: 1447533
Осталось прочитать: 1197533
Осталось прочитать: 947533
Осталось прочитать: 697533
Осталось прочитать: 447533
Осталось прочитать: 197533
CPU times: user 15min 25s, sys: 5.81 s, total: 15min 31s
Wall time: 15min 34s


Дальше все аналогично

In [15]:
max_review_len = 1
sum_review_len = 0
for x in encoded_train_google:
    sum_review_len += len(x)
    if len(x) > max_review_len:
        max_review_len = len(x)
avg_review_len = sum_review_len/len(encoded_train_google)
padded_train_google = preprocessing.sequence.pad_sequences(encoded_train_google
                                                    , maxlen = round(avg_review_len)
                                                    , padding='post')


In [16]:
padded_train_google = preprocessing.sequence.pad_sequences(encoded_train_google
                                                        , maxlen = round(avg_review_len)
                                                        , padding='post'
                                                        )

Модели параметрами оставляем те же, что были у лучшей модели на нашем W2V

In [38]:
#Linear stack of layers.
RNN_model_google = keras.Sequential()
RNN_model_google.add(layers.Embedding(input_dim=vocab_size_google
                               , output_dim=embedding_size_google
                               , input_length=round(avg_review_len)
                               , weights=[embedding_matrix_google]
                               , trainable=False
                               , mask_zero=True))
RNN_model_google.add(layers.Bidirectional(layers.SimpleRNN(units=250, activation='relu')))
RNN_model_google.add(layers.Dense(units=5, activation='softmax'))
RNN_model_google.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001, clipnorm=0.5)
                  , loss='sparse_categorical_crossentropy'
                  , metrics=['accuracy'])

RNN_model_google.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 82, 300)           900000000 
_________________________________________________________________
bidirectional_1 (Bidirection (None, 500)               275500    
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2505      
Total params: 900,278,005
Trainable params: 278,005
Non-trainable params: 900,000,000
_________________________________________________________________


Теперь пошла тренировочка

In [32]:
%%time
RNN_model_google.fit(padded_train_google, np.asarray(train_y)-1
              , epochs=5
              , batch_size=1024
              , verbose=1
              , validation_data = (padded_valid_google[:50000], np.asarray(valid_y)[:50000]-1)
              , class_weight=class_weights
              , workers=10
              , use_multiprocessing=True)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train on 1188273 samples, validate on 50000 samples
Epoch 1/5
1188273/1188273 [==============================] - 1159s 975us/sample - loss: 0.6900 - accuracy: 0.6551 - val_loss: 0.6799 - val_accuracy: 0.6629
Epoch 2/5
1188273/1188273 [==============================] - 1151s 969us/sample - loss: 0.6877 - accuracy: 0.6567 - val_loss: 0.6794 - val_accuracy: 0.6601
Epoch 3/5
1188273/1188273 [==============================] - 1147s 965us/sample - loss: 0.6770 - accuracy: 0.6615 - val_loss: 0.7255 - val_accuracy: 0.6293
Epoch 4/5
1188273/1188273 [==============================] - 1148s 966us/sample - loss: 0.6697 - accuracy: 0.6641 - val_loss: 0.6845 - val_accuracy: 0.6636
Epoch 5/5
1188273/1188273 [==============================] - 1147s 966us/sample - loss: 0.6639 - accuracy: 0.6666 - val_loss: 0.6878 - val_accuracy: 0.6551
CPU times: user 9h 43min 49s, sys: 4h 35min, total: 14h 18min 50s
Wall time: 1h 35min 53s


In [36]:
RNN_model_google.save_weights('Models/RNN_model_google_relu_adam_0.001_10epochs', save_format='tf')

In [39]:
#RNN_model_google.load_weights('Models/RNN_model_google_relu_adam_0.001_5epochs')

Фух. Ну, что-то кажется, мы не сильно лучше прошлой модель получили. Разве что обучение быстрее и сама модель более гибкая (в плане того, что слов то в этом W2V определенно больше).

In [21]:
%%time
[encoded_valid_google, valid_y] = my_encoding_reviews(Google_W2V.vocab, valid_ind)

Осталось прочитать: 1447533
Осталось прочитать: 1197533
Осталось прочитать: 947533
Осталось прочитать: 697533
Осталось прочитать: 447533
Осталось прочитать: 197533
CPU times: user 2min 29s, sys: 6.48 s, total: 2min 36s
Wall time: 2min 46s


In [22]:
padded_valid_google = preprocessing.sequence.pad_sequences(encoded_valid_google
                                                        , maxlen = round(avg_review_len)
                                                        , padding='post')

In [34]:
# здесь я фиксировал результаты обучения
results_google = pd.DataFrame([], columns=['description','epochs', 'train_loss', 'train_accuracy', 'valid_loss', 'valid_accuracy'])
results_google.loc[len(results_google)] = ['relu + adam (lr=0.001)', 5, 0.7005, 0.6503, 0.8629,0.6446]
results_google.loc[len(results_google)] = ['relu + adam (lr=0.001)', 10, 0.6639, 0.6666, 0.8706,0.6405]

In [35]:
results_google

,description,epochs,train_loss,train_accuracy,valid_loss,valid_accuracy
0,relu + adam (lr=0.001),5,0.7005,0.6503,0.8629,0.6446
1,relu + adam (lr=0.001),10,0.6639,0.6666,0.8706,0.6405


Ну, особо сильно чему-то новому модел за вторые 5 эпох не научилась. За исключением подгона под треин. Поэтому, остановимся на первой - 5 эпоховой.

In [40]:
loss, accuracy = RNN_model_google.evaluate(padded_valid_google, np.asarray(valid_y)-1, verbose=1)

254630/254630 [==============================] - 177s 696us/sample - loss: 0.8629 - accuracy: 0.6446


In [41]:
%%time
valid_pred_google_ = RNN_model_google.predict(padded_valid_google)

CPU times: user 13min 34s, sys: 8min 24s, total: 21min 58s
Wall time: 2min 41s


In [42]:
valid_pred_google = np.argmax(valid_pred_google_, axis=1)

In [43]:
print(confusion_matrix(valid_y, valid_pred_google+1, labels=np.arange(1,6)))

[[ 10806   1821    991    249   1739]
 [  4729   3996   3629    933   1955]
 [  2374   2960  11188   6766   7026]
 [  1337    805   6455  18140  30666]
 [  2169    486   2790  10608 120012]]


In [46]:
foo = 0
for i in range(5):
    for j in range(5):
        if np.abs(i-j)>1:
            foo += confusion_matrix(valid_y, valid_pred_google+1, labels=np.arange(1,6))[i][j]
print("Доля классификацей с ошибкой оценки не более чем на 1: {} %".format(round(100*(1-foo/len(valid_ind)), 2)))

Доля классификацей с ошибкой оценки не более чем на 1: 91.02 %


In [44]:
print(classification_report(valid_y, valid_pred_google+1, labels=np.arange(1,6)))

              precision    recall  f1-score   support

           1       0.50      0.69      0.58     15606
           2       0.40      0.26      0.32     15242
           3       0.45      0.37      0.40     30314
           4       0.49      0.32      0.39     57403
           5       0.74      0.88      0.81    136065

    accuracy                           0.64    254630
   macro avg       0.52      0.50      0.50    254630
weighted avg       0.62      0.64      0.62    254630

